In [11]:
import torch
import numpy as np
from sklearn.linear_model import LinearRegression
from model import NNLRModule, NNtraining, seperating

## 1. Load the data

In [12]:
x = np.genfromtxt('data/x_train.csv', delimiter=',')
y = np.genfromtxt('data/y_train.csv', delimiter=',').reshape(-1,1)

In [13]:
x_test = np.genfromtxt('data/x_test.csv', delimiter=',')
y_test = np.genfromtxt('data/y_test.csv', delimiter=',').reshape(-1,1)

In [14]:
lr_coef = np.genfromtxt('summary_statistics/beta_mean.csv', delimiter=',').reshape(-1,1)
lr_std = np.genfromtxt('summary_statistics/beta_std.csv', delimiter=',').reshape(-1,1)

## 2. Calibrate the summary statistics

In [15]:
######
print('---- performance of empirical prior')
lr_train = LinearRegression(fit_intercept=False).fit(x, y); mse = np.mean((y_test - lr_train.predict(x_test))**2); pve = 1 - np.var(y_test - lr_train.predict(x_test)) / np.var(y_test)
print('MSE: '+str(mse)+ '; PVE: '+str(pve))
print('---- performance of external prior')
y_pred = np.matmul(x_test, lr_coef)
mse = np.mean((y_test - y_pred)**2); pve = 1 - np.var(y_test - y_pred) / np.var(y_test)
print('MSE: '+str(mse)+ '; PVE: '+str(pve))
print('---- performance of calibrated external prior')
lr_calibrate = LinearRegression().fit(np.matmul(x, lr_coef), y); lr_coef = lr_coef * lr_calibrate.coef_[0][0]; lr_std = lr_std * lr_calibrate.coef_[0][0]; y_pred = np.matmul(x_test, lr_coef)
mse = np.mean((y_test - y_pred)**2); pve = 1 - np.var(y_test - y_pred) / np.var(y_test)
print('MSE: '+str(mse)+ '; PVE: '+str(pve))
#####

---- performance of empirical prior
MSE: 2.0318041518294794; PVE: -0.5485327449680681
---- performance of external prior
MSE: 1.5255735233867924; PVE: -0.15990745527944372
---- performance of calibrated external prior
MSE: 1.331835053602444; PVE: 0.04212891864556545


## 3. NN training

In [6]:
x = torch.tensor(x, dtype=torch.float); y = torch.tensor(y, dtype=torch.float)
x_test = torch.tensor(x_test, dtype=torch.float); y_test = torch.tensor(y_test, dtype=torch.float)
x_train, y_train, x_val, y_val = seperating(x, y)

### 3.1 NN with main effect prior

In [7]:
batch_size = 100
num_epoch = 3000
num_nodes = [20, 10]
learning_rate=0.001

reg_weight_main = 0.05
reg_weight_nn = 0.05
reg_weight_info = 0.01

In [8]:
torch.manual_seed(0)
model = NNLRModule(x.shape[1], num_nodes)
NN = NNtraining(model, learning_rate=learning_rate, batch_size=batch_size, num_epoch=int(num_epoch), reg_weight_main=reg_weight_main, reg_weight_nn=reg_weight_nn, reg_weight_info=reg_weight_info, use_early_stopping = True)
NN.training(x_train, y_train, x_val, y_val,  main_effect = lr_coef,  main_effect_std = lr_std, marginal = False)
mse, pve = NN.build_evaluation(x_test, y_test)
print(pve)

Compute the gram matrix
Compute the gram matrix done!
>>> Epoch   250/ 3000 | train_mse=0.69119 | val_mse=0.91281 | train_pve=0.47684 | val_pve=0.20437
tensor(0.1391)


### 3.2 NN without main effect prior

In [9]:
reg_weight_info = 0.0
torch.manual_seed(0)
model = NNLRModule(x.shape[1], num_nodes)
NN = NNtraining(model, learning_rate=learning_rate, batch_size=batch_size, num_epoch=int(num_epoch), reg_weight_main=reg_weight_main, reg_weight_nn=reg_weight_nn, reg_weight_info=reg_weight_info, use_early_stopping = True)
NN.training(x_train, y_train, x_val, y_val, main_effect = lr_coef,  main_effect_std = lr_std, marginal = False)
mse, pve = NN.build_evaluation(x_test, y_test)
print(pve)

Compute the gram matrix
Compute the gram matrix done!
>>> Epoch   317/ 3000 | train_mse=0.62534 | val_mse=0.96895 | train_pve=0.52669 | val_pve=0.16121
tensor(0.1018)
